In [1]:
!pip install datasets

!pip install 'accelerate>=0.26.0'

!pip install -U sentence-transformers

  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.9 MB/s eta 0:00:00
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.17
    Uninstalling multiprocess-0.70.17:
      Successfully uninstalled multiprocess-0.70.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.3 requires dill>=0.3.9, but you have dill 0.3.8 which is incompatible.
pathos 0.3.3 requires multiprocess>=0.70.17, 

In [2]:
from sentence_transformers import SentenceTransformer, CrossEncoder
embeddings_model_source = 'BAAI/bge-base-en-v1.5'
cross_encodings_model_source = 'cross-encoder/ms-marco-MiniLM-L6-v2'
embeddings_model = SentenceTransformer(embeddings_model_source)
# cross_encodings_model = CrossEncoder(cross_encodings_model_source)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
import boto3
import json
import io
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from datasets import Dataset
import logging
from sentence_transformers import LoggingHandler
import random
from sentence_transformers import InputExample, losses
from sentence_transformers.evaluation import TripletEvaluator
from torch.cuda.amp import autocast


logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
# Now run embeddings_model.fit(...) and you’ll see training loss printed during training.


In [12]:
# === Config ===
s3_bucket = 'big-cloud-country-sandbox'
s3_key = 'fine-tune-data/embeddings/embedding_data_hard_negs_4.jsonl'  # or train.jsonl
local_model = 'BAAI/bge-base-en-v1.5'
output_path = './finetuned-bge-nuclear-4-3'
batch_size = 32
num_epochs = 5
grad_accumulation_steps = 4

In [13]:
# === Step 1: Load training data from S3 ===
s3 = boto3.client('s3', region_name='us-east-2')
obj = s3.get_object(Bucket=s3_bucket, Key=s3_key)
lines = obj['Body'].read().decode('utf-8').splitlines()

# Shuffle lines
random.seed(42)  # For reproducibility
random.shuffle(lines)

In [14]:
# === Step 2: Split into training and validation sets ===
split_ratio = 0.8
split_index = int(len(lines) * split_ratio)
train_lines = lines[:split_index]
val_lines = lines[split_index:]

In [15]:
# === Step 3: Convert to InputExamples ===
train_examples = []
for line in train_lines:
    record = json.loads(line)
    train_examples.append(InputExample(texts=[
        record['query'], record['positive'], record['negative']
    ]))

val_anchors, val_positives, val_negatives = [], [], []
for line in val_lines:
    record = json.loads(line)
    val_anchors.append(record['query'])
    val_positives.append(record['positive'])
    val_negatives.append(record['negative'])

In [16]:
# === Step 4: Set up train dataloader ===
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# === Step 5: Define loss function ===
# train_loss = losses.TripletLoss(model=embeddings_model)
train_loss = losses.MultipleNegativesRankingLoss(model=embeddings_model)


# === Step 6: Set up evaluator ===
evaluator = TripletEvaluator(
    val_anchors,
    val_positives,
    val_negatives,
    name='validation',
    batch_size=8,
)
evaluator.write_csv = True

In [17]:
embeddings_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    # gradient_accumulation_steps=4,
    use_amp=True,
    epochs=num_epochs,
    warmup_steps=100,
    show_progress_bar=True,
    output_path=output_path,
    evaluation_steps=200  # or whatever makes sense for your training speed
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Validation Cosine Accuracy
200,No log,No log,0.927644
270,No log,No log,0.930891
400,No log,No log,0.931354
540,0.566200,No log,0.935529
600,0.566200,No log,0.935529
800,0.566200,No log,0.937384
810,0.566200,No log,0.935529
1000,0.242900,No log,0.935993
1080,0.242900,No log,0.935529
1200,0.242900,No log,0.935529


2025-04-15 21:00:34 - TripletEvaluator: Evaluating the model on the validation dataset in epoch 0.7407407407407407 after 200 steps:
2025-04-15 21:00:42 - Accuracy Cosine Similarity:	92.76%
2025-04-15 21:00:42 - Save model to ./finetuned-bge-nuclear-4-3
2025-04-15 21:01:01 - TripletEvaluator: Evaluating the model on the validation dataset in epoch 1.0 after 270 steps:
2025-04-15 21:01:08 - Accuracy Cosine Similarity:	93.09%
2025-04-15 21:01:08 - Save model to ./finetuned-bge-nuclear-4-3
2025-04-15 21:01:55 - TripletEvaluator: Evaluating the model on the validation dataset in epoch 1.4814814814814814 after 400 steps:
2025-04-15 21:02:03 - Accuracy Cosine Similarity:	93.14%
2025-04-15 21:02:03 - Save model to ./finetuned-bge-nuclear-4-3
2025-04-15 21:02:40 - TripletEvaluator: Evaluating the model on the validation dataset in epoch 2.0 after 540 steps:
2025-04-15 21:02:48 - Accuracy Cosine Similarity:	93.55%
2025-04-15 21:02:48 - Save model to ./finetuned-bge-nuclear-4-3
2025-04-15 21:03:0